In [2]:
import re
import pandas as pd
import numpy as np
import datetime, time
from soyspacing.countbase import CountSpace
import random
from soynlp.normalizer import *
import requests
from bs4 import BeautifulSoup

# 수집한 리뷰 데이터 열기, timestamp 추가

In [36]:
numbers = [0,30,60,90,120,150]
DF_List = []

for num in numbers:
    DF = pd.read_excel('review_%s.xlsx'%(num))
    DF_List.append(DF)
    
    
WHOLE_REVIEW_DF = pd.concat(DF_List)

In [40]:
print("Shape of review table\n",np.shape(WHOLE_REVIEW_DF))

Shape of review table
 (744211, 4)


In [123]:
WHOLE_REVIEW_DF.to_excel('전체리뷰.xlsx')

In [60]:
WHOLE_REVIEW_DF.columns = ['평점','시간','리뷰','이름']

print("Number of games\n",np.shape(np.unique(WHOLE_REVIEW_DF.이름)))

Number of games
 (178,)


In [84]:
def to_time_obj(string):
    time_list = [int(re.search('(\d*)년 (\d*)월 (\d*)일',string).group(x)) for x in range(1,4)]
    return datetime.datetime(*time_list)

WHOLE_REVIEW_DF.시간 = WHOLE_REVIEW_DF.시간.apply(to_time_obj)
WHOLE_REVIEW_DF['timestamp'] = WHOLE_REVIEW_DF['시간'].apply(lambda x : time.mktime(x.timetuple()))

In [53]:
WHOLE_REVIEW_DF = pd.read_excel('전체리뷰.xlsx')

# 스팸리뷰 제거

In [74]:
# 일단 null type을 모두 없애야함.
review = WHOLE_REVIEW_DF.리뷰
def null_to_none(string):
    if string != string or type(string) == type(1):
        return ''
    else:
        return string
review = review.apply(null_to_none)

In [75]:
# 간단한 예시로는 이런식으로 일부분의 스팸을 확인가능.
def compile_spam(regex):
    compile_df = review.str.contains(regex,regex=True)
    print(review[compile_df].head(30))
    review[compile_df] = '스팸메시지'
    
compile_spam('▶.*◀')

3311     셱◔주소 ▶ 주소챵에 → Ν A ν E R ２ ３ ,К r ◀ 이렇게 핫한 채팅어플...
3313     Τ ▶ 주소챵에 → Ν A ν E R ２ ３ ,К r ◀ 이렇게 핫한 채팅어플 써본...
3336     K ▶ 주소챵에 → Ν A ν E R ２ ３ ,К r ◀ 이렇게 핫한 채팅어플 써본...
3338     Η ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
3339     Ζ ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
3340     Τ ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
3365     Е ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
3366     Ｆ ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
3396     +　쌕　s　e　x　파 ▶ S　e　x 8 9 8 9 , С　Ο　M ◀ 이렇게 핫한 채...
3459     ħ ▶ 주　소　창에 → N　A　V　E　R 2　3 . k　r ◀ 이렇게 핫한 채팅어플...
8888     K ▶ 주소챵에 → Ν A ν E R ２ ３ ,К r ◀ 이렇게 핫한 채팅어플 써본...
8891     Ζ ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
8892     Τ ▶ S　e　x 8 9 8 9 , с　о　м ◀ 이렇게 핫한 채팅어플 써본거는 오...
8913     +　쌕　s　e　x　파 ▶ S　e　x 8 9 8 9 , С　Ο　M ◀ 이렇게 핫한 채...
8918     Æ ▶ S　e　x 8 9 8 9 , C　0　M ◀ 이렇게 핫한 채팅어플 써본거는 오...
16574    섹◔파톡 ▶ 주소챵에 → Ν A ν E R ２ ３ ,К r ◀ 이렇게 핫한 채팅어플...
16577    셱◔주소 ▶ 주소챵에 → Ν A ν E R ２ ３ ,К r ◀ 이렇게 핫한 채팅어플.

In [76]:
compile_spam('누드딜러')

153      [kew19.com]누드딜러와 화끈한 한판승부 No.1 온라인카지노,해외스포츠 온라...
56268    [kew19.com]누드딜러와 화끈한 한판승부 No.1 온라인카지노,해외스포츠 온라...
64600    [kew19.com]누드딜러와 화끈한 한판승부 No.1 온라인카지노,해외스포츠 온라...
Name: 리뷰, dtype: object


In [35]:
# 비슷하게 특수문자들에 둘러싸인 글들은 스팸일 가능성이 높다고 판단했음.
# 특수문자들을 모아놓은 사이트들에서 특수문자 목록을 가져와서 확인해보자.

url = 'https://m.blog.naver.com/yeojh1/221165758547'
soup = BeautifulSoup(requests.get(url).text,'html.parser')
selected = soup.select('div.post_ct > p')

special_char = ""
for e in selected[10:]:
    special_char += e.get_text()

special_char = list(set(special_char))
print("특수문자 개수 : %s"%(len(special_char)))

특수문자 개수 : 890


In [27]:
def print_spam(regex):
    try:
        compile_df = review.str.contains(regex,regex=True)
        if 1 < len(review[compile_df]) < 1000:
            print('---------------------------------------')
            print('regex=%s \n'%(regex))
            print(*review[compile_df].head(10).values,sep='\n')
    except:
        return

for e in special_char:
    print_spam('%s.+%s'%(e,e))
    
# 진짜 이상한 리뷰가 많다.
# 그런데 특별히 스팸으로 판단되는 리뷰는 거의 보이지 않음.

---------------------------------------
regex=ั.+ั 

ะุดใสฝ~บหช้ชพบนฃีบภนัภยับภยถยัถาาัำสันสไบั
กขหชักฃัก
เข้าเกมไม่ได้ครับไม่รู้ว่าเปันไร
เขารายงานมั่วหักคะแนน ตรวจสอบหน่อยสิ ตาไม่มีเหรอ ฆ่า5 ตาย4 ช่วย7 MVP มันบอกตั้งใจภูกฆ่า งงเลย
ไม่ได้ฟังเพลงดีไอ้สัสเอ้ย กูรอฟังเพลงหลายรอบละนะ แม่งมีแต่เสียงปี้ดๆๆ
---------------------------------------
regex=！.+！ 

오랜만에 눈 귀 손이 모두 즐거운 게임이었습니다. 老板加油吧！游戏很厉害！非常公平的游戏，付出努力结果一定回来的。真喜欢。韩语翻译还好，喜欢喜欢。
Good job！！！！
本当に面白いゲームですね。！！！
最高！！！
---------------------------------------
regex=∩.+∩ 

와우 완전 슈퍼 그뤠잇 (∩_∩)(∩_∩)(∩_∩)(∩_∩)(∩_∩)(∩_∩)(∩_∩) !!!!!!
넘넘 재밌어요! 불편한 점이 있다고 하시는 분들이 종종 게시는데 저는그런거 못느끼는거 같아요∩'ω'∩ 제가 재넌 보면서 폰으로 재넌이 하는게임이랑 비슷한거 해보고 싶었거든요! 근데 이게 저한테 잘 맞고 즐거운거 같아요! 무조건! 별 5개!!
와 오랜만에 맞는게임 찾았네 한마디로 표현하면..(∩_∩)개꿀잼
재밌어요∩'ω'∩
왠지 전보다 더재미있어졌어요!!!앞으로더많은발전부탁드릴게요!!(참고로전이게임이저의최애게임입니당!!)♡♡♡(∩0∩)
이거 퇴마고에서 코인으로 350코인 준다고 해서 깔아서5레벨까지 갔는데 진짜 주더라고요..글구별1만 주는분들. 이렇게 잘만든 앱을 왜 별1개만 주나요 전 5개 줍니다∩'ω'∩리뷰 작성하고 10레벨 도전!!!!!너무 재밌어요
욕하지마세요.욕하는 사람달 모야?넘잼.꾸르으잼 ㅇㅈ?ㅎㅎ살짝 지러하기는 한데 그래도 힐링되고 

---------------------------------------
regex=↓.+↓ 

별 한개도 주기싫네×이거 사기야?진짜 핵노잼↓↓↓😠😡😡😡😡😡
↑↓↓↓
집진는게기재미있지만게임은 어려워용↑↓↑↑↓↑
---------------------------------------
regex=∀.+∀ 

다좋은데 10게의 문제점 1.야하다 2.야하다 3.야하다 4.  광고가 너무 야하다 5.다 야하다 그리고6.ㄱㅅ이 너무 크다 7ㄱㅅ이 엄청크다 7.ㄱㅅ이 너무크다 8.ㄱㅅ이 너무크다 9.ㄱㅅ이 너무크다 그리고 대망의 10번 10.........광고는 쫌 야하지 않게 해주시길ㅠㅠ 제발 그 광고 봐서 꿈에서 여케들이 나한테 가ㅅ을 계속 내밀어요 잠자는게 무서워요ㅠㅠ 제발좀 이것만 고쳐주시면 감사하겠어요!!!! 그리고 케들이 넘 예뻐요!♥♥♥♥♥♥♥♥♥♥♥(나 남자) 여케들은 진짜로 대박 완전 예뻐!!!(≥∀≤)/ (≥∀≤)/ (≥∀≤)...<div class="cQj82c"><button class="LkLjZd ScJHi OzU4dc " jsaction="click:TiglPc" jsname="gxjVle">전체 리뷰</button></div>
말이 필요 없어요 그냥 해보시면 왜 계속 하게 되는지 알 수 있음요 (≥∀≤)/ (≥∀≤)/ (≥∀≤)/
정말 재미있어요. 처음하는데 전혀 부담이 안가네오. 최곱니다 (≥∀≤)/ (≥∀≤)/ (≥∀≤)/
장비를 자주 갈아끼워야 할 정도로 장비 지원 잘해주는거 실화임? 진짜 대박임 (≥∀≤)/ (≥∀≤)/ (≥∀≤)/
업데이트 해 주새요.♥♥♥♥♥★★★★★(^o^)b (^o^)b (^-^)v (^-^)v (^ 3^) (^ 3^) (^-^)v(≥∀≤)/ (≥∀≤)/ (^ 3^) (^ 3^) (^ 3^)
---------------------------------------
regex=´.+´ 

......................./´¯/) ....................,/¯../ ..................

---------------------------------------
regex=《.+《 

아주 좋은 게임이다 솔직히 말해서 나는 보기만 하는 게임이다   전략 그딴거없다 그냥 예쁜 인형들이 알아서 싸우는거  《이제 케릭터스킬누르는것도 귀찮아》걱정없다 자동스킬이있으니깐 《이제  싸우는것도 귀찮아》 걱정없다 자동훈련이 싸우는것도없이 그냥 시간만지나면 케릭터 레벨이 올라갈수있으니깐 생각해보니깐 왜재미없는지 알거같음 게임자체가 싸우는거 보는거 앞뒤로 움직이는 거밖에없음
저기요, 제작자님, 제가 요즘에 냥코대전쟁을 많이 하는 데요. 제가 별 4개만드린 이유는 냥코대전쟁에 《친구 추가》하고 《친구와의 배틀》 등 이 있으면 좋겠다는 생각이 들어서 말씀 드린 거예요. 또 울슈레가 나올 확률을 올러주세요. 이것 만 조금 수정 해주시면 감사하겠습니다. 연락부탁드립니다.
《♡※꿀잼※♡》 《☆완전☆》 (♡♡꿀잼♡♡) &lt;&amp;임&amp;&gt; [♧이걸♧] {♡본 사람은♡} (@다운로드@) 해보세요.
---------------------------------------
regex=◕.+◕ 

。◕‿◕ 。๑·̑◡･̑๑๑♥‿♥๑
요번에 처음 깔고 리뷰 남겨요:) 캐릭터, 스토리 그외 등등 탄탄하고 대체적으로 밸런스도 맞아서 재밌게 플레이 했는데 처음 할때는 처음이니깐 렉이 좀 걸리나 보다 했거든요·· 근데 몇일이 지나도 렉이 점점 심해지거나 그대로이지 렉이 나아지질 않아요 용량 때문에 그런가 해서 앱 몇개를 삭제 해봐도 그대로구요 저만 그런건지 몰라도 이 문제만 개선 되면 평점 만점 게임일거 같아요 (◕◡◕)
٩(◕ᗜ◕)و
---------------------------------------
regex=→.+→ 

우선 모션이 사냥하고나서 가까히도 안가고 그냥 아이템을 손으로 휘적거리며 얻네요 그리고 세 개의왕국에서 설명이 위나라를 청나라가 정복해서 끝났다고 하는데 위나라는 진나라가되고요 진나라→ 위진남북조 →수나라 →당나라→ 명나라 →청나라 이렇게 늦게 건국된 국가가

---------------------------------------
regex=×.+× 

일반 기지는 완전나아요 아니 근데 장인기지에 보석이 1일에 한번식나오는거 실화냐? 아 그리고 장인 기지에서 유닛 업그래이드해도 완파를 못해! 씨× 아!!! 미친 씨×색× 아!!!!!!!!!
모마가 많이 좋아졌어요!! 건의좀 하고싶은데.. 현재 모마는 8칸×8칸이잖아여 ? 만약 12칸×12칸(대규모판)만들면 어떨까요? 예를 들어 현재 8칸은 무인도면, 12칸도 무인도로 만드는거에요! 재밌을것같아요!
859스테이지 하다가 분노가 치밀어서 도저히 안되겠다 게임 탈퇴한다 애니팡2가 왜 부진한지를 ×잡고 곰곰히 생각해 보거라. ×새들아
야 이거개임 만든사람누구야! 이런 시×색×
야이××××야 와이파이가잘 안돼자나 잇×××× 야
레어뽑기에서 루피×2,상디×2,조로,다른조로,나미,우솝,마르코,샹크스...이렇게 나왔는데 좋게 나온건지...
이런××××××××××××××××××
강화 확률이 ××××××××O×××OO×××××××O××××OOOOO  덱은 쓰레기 중복만 두고ㅠㅜ
별-99999999999999999999999999999999999999999999999999999999×999999999999×99999999999×9999×9×9×9×9×9×9×9×9×9×9×9×99×9×9999999999999999999999×99999999999999×9×9×9×9×9×9×9×9999999999999999999999999999×9×9×9×9×9×9×9×9999999×99999×99999×999×999999999999×99999×99×9×9×9×9999999개
×××××××
---------------------------------------
regex=♥.+♥ 

정말 최고에용~♥♥♥♥♥♥♥
♥♥♥♥♥
。◕‿◕ 。๑·̑◡･̑๑๑♥‿♥๑
꿀잼이에요!!!!!!!!!!!!!!!김사합니다♥♥♥♥♥♥♥♥
모두의마블 정말 잘만드셧어요!♥ 운영자님 사랑합니다♥
몬스터 길드리기 추천해요 개추천 ♥♥♥

---------------------------------------
regex=↑.+↑ 

호롤↑호롤↑호로로롤↓호롤-호로롤-호롤로롤
스킬은당영히 멋닜고 욕할정도로 그레픽↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑
집진는게기재미있지만게임은 어려워용↑↓↑↑↓↑
심각↓하↑다↑
---------------------------------------
regex=☜.+☜ 

✩✪✫✬✭✮✯✰✱✲✳✴✵✶✷✸✹✺ ✻✼✽✾✿❀❁❂❃❄❅❆❇❈❉❊❋❍ ❏❐❑❒ ❖ ❘❙❚❛❜❝❞ ❡ ❢❣❤❥❦❧ ❶❷❸❹❺❻❼❽❾❿ ➀➁➂➃➄➅➆➇➈➉ ➊➋➌➍➎➏➐➑➒➓ ➔ ➘➙➚➛➜➝➞➟➠➡➢➣➤➥➦➧➨ ➩➪➫➬➭➮➯ ➱➲➳➴➵➶➷➸➹➺➻➼➽➾ ★☆ ☉ ☎☏ ☜ ☞ ☬ ♀ ♂ ✁✂✃✄ ✆✇✈✉✌✍✎✏✐✑✒ ☉ ☎☏ ☜ ☞ ☬ ♀♂ ♔♕♖♗♘♙♚♛♜♝♞ ლ ♛ ๑ ✧ இ ஈ ஊ எ ஏ ஐ ஒ ஓ ஔ ◌ஂ ஃ ◌ா ◌ி ◌ௗ ங ஜ ஞ ண ந ன ம ய ற ல ள ழ வ ஸ ஹ ௧ ௨ ௩ ௪ ௫ д ╬ ☉ ღ ❤ ஐ◌ி Δ ❤ˇټ ๑...<div class="cQj82c"><button class="LkLjZd ScJHi OzU4dc " jsaction="click:TiglPc" jsname="gxjVle" style='font-family: Roboto, "Noto Sans KR", sans-serif;'>전체 리뷰</button></div>
마녀와 베이비 드래곤 완전 좋음 해골군대와 호그 라이더(?)함치면(?)더 좋음!o(^-^)o ♬♪♩☜☜☜♥★♥★💎💎💎👈👈
---------------------------------------
regex=□.+□ 

3인칭조아하시는분들 이거 추천각임ㄴ당 □ㅅ□
옆동에 □□□마블 보다 주작이...
□■□■
이것은 □□□이다
뭔데 왜 넷□블의 이□만루와 비슷한게임 인데
이 게임의 레벨 올리기 난이도:■□□□□
---------------------------------------
regex=§.+§ 

왜 이롷게 옵대이트 해야 하는데 업데이트가 안뜨나 요/§§§§§§§§§

---------------------------------------
regex=д.+д 

아 개극혐 이야 The game sucks. Сдуьв вдвба ыдсду вбвьу
игра Соединение Медленно Решения?
Би сайн ойлгохгүй байна гадаад хэл болохоор тэр үү


In [77]:
# 몇개의 리뷰만 제거해보자.
compile_spam('현.?금.?카.?지.?노')

3347      현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
8903      현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
16644     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
26817     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
37771     현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
42673     현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
56107     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
79997     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
11152     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
58647     현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
67645     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
82248     현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
110579    현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
6306      현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
9538      현금카◍지노 🔥 M Χ M 2 １ ̨ℂ 0 Ⅿ 🔥 이렇게 핫한 현금어플 써본거는 오...
12325     현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇게 핫한 현금어플 써본거는 오...
40593     현금카◉지노 🎈 M Χ M 2 １ ̨ℂ 0 Ⅿ 🎈 이렇

In [78]:
compile_spam('섹.{1}파')

16718    섹◔파톡 ❤ Ⅽ Ｖ C 9 ３ .ⅽ ₀ ⅿ ❤ 이렇게 핫한 채팅어플 써본거는 오랜만...
30840    섹 파 새 끼 애 미 쳐 뒤 진 년 이 구 요~         섹 파 새 끼 애 미...
67664    섹◔파톡 ❤ Ⅽ Ｖ C 9 ３ .ⅽ ₀ ⅿ ❤ 이렇게 핫한 채팅어플 써본거는 오랜만...
30655                               ㅇ? 왜 경기 이름이 광란에 섹스파티임?
Name: 리뷰, dtype: object


# 약간의 전처리 + 띄어쓰기 교정

In [5]:
# 이전에 지식인 데이터를 통해서 이미 학습하긴했음.
model = CountSpace()
model.load_model('../datamining/model_soyspacing.txt',json_format=False)

In [79]:
# 데이터를 약간 전처리하고 우리 데이터에도 모델을 학습하기로함.
def remove_script(string):
    return re.sub('<(.*)>','',string)
def remove_strange(string):
    return re.sub('[A-Za-z;&-]|전체리뷰','',string)

review = review.apply(remove_script)
review = review.apply(remove_strange)
# emoticon_normalize, repeat_normalize : soyspacing 함수
# 반복되는 이모티콘과 글자를 축약시켜주는 역할.
review = review.apply(emoticon_normalize)
review = review.apply(repeat_normalize)
corpus = '\n'.join(review.values)
print(len(corpus))

20239346


In [81]:
with open('soyspacing_feed.txt','w') as fileobj:
    fileobj.write(corpus)
model.train('soyspacing_feed.txt')
model.save_model('fname=playstore_soyspacing_model.txt',json_format=False)

all tags length = 4191173 --> 4133812, (num_doc = 744210)

In [44]:
model = CountSpace()
model.load_model('fname=playstore_soyspacing_model.txt',json_format=False)

In [81]:
def correct(string):
    # 길이짧은건 교정하느라 시간날릴것같아서 안했음.
    if len(string) >= 5:
        sent_corrected, tags = model.correct(
            doc=string,
            verbose=False,
            force_abs_threshold=0.3,
            nonspace_threshold=-0.3,
            space_threshold=0.3,
            min_count=10)
        # 1000개중에 한개꼴로 프린트하게 만듬.
        if random.uniform(0,1) < 1e-3 and len(string) >= 20:
            print('----------------------')
            print('교정 이전 len = %s \n'%(len(string)),string)
            print('교정 이후 len = %s \n'%(len(sent_corrected)),sent_corrected)
        
        return sent_corrected
    
    else:
        return string
    
%time corrected_review = review.apply(correct)

# 이상한 리뷰가 많긴했지만 정상적인 리뷰가 압도적으로 많으므로 이모티콘,특수문자 제거를 하지 않아도 될듯.

----------------------
교정 이전 len = 70 
 아니 제가 11홀까지 키운 계정이날라갔고 다시 1홀입니다 여기다가 현질만 37만원했는데 계정날라가니까 화가나네요 복구해 주세요
교정 이후 len = 71 
 아니 제가 11홀까지 키운 계정이 날라갔고 다시 1홀입니다 여기다가 현질만 37만원했는데 계정날라가니까 화가나네요 복구해 주세요
----------------------
교정 이전 len = 50 
 오늘 업데이트를 하고나서 멀티플레이 별보너스를 모을수가 없습니다. 수정과 보상 부탁드립니다
교정 이후 len = 50 
 오늘 업데이트를 하고나서 멀티플레이 별보너스를 모을수가 없습니다. 수정과 보상 부탁드립니다
----------------------
교정 이전 len = 74 
 아니리그돌리는데왜실버가골드를만나냐고요.실버는실버끼리.골드는골드끼리아니에요?.그래도모마에서나누어준탑10종캐릭으로루시드얻었네여그건감사해요
교정 이후 len = 81 
 아니리그돌리는데 왜실버가골드를 만나냐고요.실버는실버끼리.골드는 골드끼리아니에요?.그래도모마에서 나누어준탑10 종캐릭으로 루시드얻었네여 그건감사해요
----------------------
교정 이전 len = 103 
 다시 깔았는데 약관동의를 안해서 게임을 종료한다는데요 뭘 동의를 안한건지 지우고 다시깔아도 똑같습니다 깔고 들어가자마자 약관동의을 하지않아 게임을 종료합니다 그러면 전 게임못하는건가요?
교정 이후 len = 103 
 다시 깔았는데 약관동의를 안해서 게임을 종료한다는데요 뭘 동의를 안한건지 지우고 다시깔아도 똑같습니다 깔고 들어가자마자 약관동의을 하지않아 게임을 종료합니다 그러면 전 게임못하는건가요?
----------------------
교정 이전 len = 73 
 담당자님 오래 간만에 게임 하려고 들어 가보니 비비안 서버의 캐릭이 거의 삭제 되고 고유장비가 다 없어저있네요ㅡㅡ 복구 부탁드립니다
교정 이후 len = 73 
 담당자님 오래 간만에 게임 하려고 들어 가보니 비비안

----------------------
교정 이전 len = 118 
 진삼국무쌍 모바일 최고임 플스로 진삼국무쌍시리즈 싹다했는데 정말 잘만들었음 모바일인데 콘솔못지않는 그리픽 게임성 시스템 좋음 특히 넥슨답지않게 과금유도도 없고 무과금으로 해도 게임 진행에 영향없음 진삼모바일 최고
교정 이후 len = 118 
 진삼국무쌍 모바일 최고임 플스로 진삼국무쌍시리즈 싹다했는데 정말 잘만들었음 모바일인데 콘솔못지않는 그리픽 게임성 시스템 좋음 특히 넥슨답지않게 과금유도도 없고 무과금으로 해도 게임 진행에 영향없음 진삼모바일 최고
----------------------
교정 이전 len = 34 
 사전예약을 했는데 사전예약 쿠폰이 안왔습니다 어떻게 된건가요?
교정 이후 len = 34 
 사전예약을 했는데 사전예약 쿠폰이 안왔습니다 어떻게 된건가요?
----------------------
교정 이전 len = 28 
 멈춤현상 랙걸리는거 어떻게좀 해야 게임을 하든지하지
교정 이후 len = 28 
 멈춤현상 랙걸리는거 어떻게좀 해야 게임을 하든지하지
----------------------
교정 이전 len = 98 
 잼있기는한데 중간 중간 튕기고 컨트롤이 안되거나 터치가 안되는 오류가 있어 종료하고 다시 접속해야하는 일이 종종 있습니다. 근데 플스에서 했던건 게임을 하니까 좋기는 하군요..
교정 이후 len = 99 
 잼있기는 한데 중간 중간 튕기고 컨트롤이 안되거나 터치가 안되는 오류가 있어 종료하고 다시 접속해야하는 일이 종종 있습니다. 근데 플스에서 했던건 게임을 하니까 좋기는 하군요..
----------------------
교정 이전 len = 44 
 으따~ 이게 그 느그전선인가 뭔가 하는것구마잉~ 쇼죠젠센말고 깡겜하자 이거야^^
교정 이후 len = 44 
 으따~ 이게 그 느그전선인가 뭔가 하는것구마잉~ 쇼죠젠센말고 깡겜하자 이거야^^
----------------------
교정 이전 len = 178 
 게임 자체는 괜찮지만 퍼블리셔가

----------------------
교정 이전 len = 32 
 오늘 갑자기 로그아웃 되더니 장수 다잃었어요. 어떻하나요?
교정 이후 len = 32 
 오늘 갑자기 로그아웃 되더니 장수 다잃었어요. 어떻하나요?
----------------------
교정 이전 len = 148 
 조조전 연의 조조남하 스토리설명부분 양양을 손에 넣운 조조는 에서 양양을 손에 넣은 조조는 으로 수정부탁드려요 + 점검할때 아무말도없이 재시도하라는 창이뜨고 다시 들어가면 또 계속 재시도하라고 창이 나와서 놀란적이 있는데 공지라도 해두시고 점검해주시면 감사하겟습니다
교정 이후 len = 149 
 조조전 연의 조조남하 스토리설명부분 양양을 손에 넣운 조조는 에서 양양을 손에 넣은 조조는 으로 수정부탁드려요 + 점검할때 아무말도없이 재시도하라는 창이 뜨고 다시 들어가면 또 계속 재시도하라고 창이 나와서 놀란적이 있는데 공지라도 해두시고 점검해주시면 감사하겟습니다
----------------------
교정 이전 len = 82 
 다 좋아졌는데.. 서신 일괄수령하구 금전 사용하게 될때 경고문 한번만 좀 나왔으면 좋겠네요.. 성 업글하다가 금전 한번 쓰게되니까 짜증이 팍...ㅠ
교정 이후 len = 82 
 다 좋아졌는데.. 서신 일괄수령하구 금전 사용하게 될때 경고문 한번만 좀 나왔으면 좋겠네요.. 성 업글하다가 금전 한번 쓰게되니까 짜증이 팍...ㅠ
----------------------
교정 이전 len = 26 
 스토리가 재미있고 게임 조작이 쉽고 재미있어요.
교정 이후 len = 26 
 스토리가 재미있고 게임 조작이 쉽고 재미있어요.
----------------------
교정 이전 len = 33 
 2017년도 인 줄... 2018 선수들이 하나도 없네요 ㅠ
교정 이후 len = 34 
 2017 년도 인 줄... 2018 선수들이 하나도 없네요 ㅠ
----------------------
교정 이전 len = 37 
 오 이런거 너무 좋은것 같아용 야구

----------------------
교정 이전 len = 41 
 시간내서 광고보기 했어도 끝까지 안본거라며 하트를 안줌 광고보기는 왜 함?
교정 이후 len = 41 
 시간내서 광고보기 했어도 끝까지 안본거라며 하트를 안줌 광고보기는 왜 함?
----------------------
교정 이전 len = 68 
 업데이트한 후로 앱을 실행시키면 첫화면에서 갑자기 꺼져버립니다. 지웠다가 다시 깔아도 증상은 똑같습니다 어떻게 해야할까요?
교정 이후 len = 68 
 업데이트한 후로 앱을 실행시키면 첫화면에서 갑자기 꺼져버립니다. 지웠다가 다시 깔아도 증상은 똑같습니다 어떻게 해야할까요?
----------------------
교정 이전 len = 24 
 아재로써 개취향저격 좋네요 파판7 생각남 ㅋ
교정 이후 len = 24 
 아재로써 개취향저격 좋네요 파판7 생각남 ㅋ
----------------------
교정 이전 len = 23 
 4주동안 무지개 다섯개나 주시고 감사합니다
교정 이후 len = 24 
 4 주동안 무지개 다섯개나 주시고 감사합니다
----------------------
교정 이전 len = 259 
 뽑기 무슨생각으로 만든거임? 5성 나오긴 나오나요 내가 운이 안 좋은걸수도 있는데 맨첨에 5성확정주는거 빼고는 5성 한번도 안 나오고 4성도 여태까지 3번도 안 나왔음 그래서 전설티켓 그 다이아 600개 주고 돌렸는데도 3성파티 ㅋㅋㅋ 장난하나요? 확률이 도데체 얼마나 되는건가요 10번 연속 돌릴때 적어도 4성 1개 확정이여야 하는거 아닌가요? 다른게임들이랑 비교해서 죄송한데 다른게임들은 다 그래요... 어떻게 확률이 이래 다른건 진짜 재밌고 좋은데요 수정좀요
교정 이후 len = 261 
 뽑기 무슨생각으로 만든거임? 5성 나오긴 나오나요 내가 운이 안 좋은걸수도 있는데 맨첨에 5성확정주는거 빼고는 5성 한번도 안 나오고 4성도 여태까지 3 번도 안 나왔음 그래서 전설티켓 그 다이아 600개 주고 돌렸는데도 3성파티 ㅋㅋㅋ

----------------------
교정 이전 len = 28 
 점심시간에 잠깐 하려고 했는데 점심시간이 사라졌어요
교정 이후 len = 28 
 점심시간에 잠깐 하려고 했는데 점심시간이 사라졌어요
----------------------
교정 이전 len = 42 
 아침부터 뉴포커로 대박터져서 너무 기분 좋네여 ㅋㅋㅋ 항상 이랬으면 ㅠㅠ !
교정 이후 len = 42 
 아침부터 뉴포커로 대박터져서 너무 기분 좋네여 ㅋㅋㅋ 항상 이랬으면 ㅠㅠ !
----------------------
교정 이전 len = 23 
 너무 재미써요 한참 피씨방 많이갔는디 ㅎㅎ
교정 이후 len = 24 
 너무 재미써요 한참 피씨방 많이 갔는디 ㅎㅎ
----------------------
교정 이전 len = 22 
 최고입니다!! 유튜브 에이스 구독해주세요
교정 이후 len = 22 
 최고입니다!! 유튜브 에이스 구독해주세요
----------------------
교정 이전 len = 38 
 야구팬이라면 강추!! 나만의구단을 만들어 선수모으는 재미도 쏠쏠하네요
교정 이후 len = 38 
 야구팬이라면 강추!! 나만의구단을 만들어 선수모으는 재미도 쏠쏠하네요
----------------------
교정 이전 len = 33 
 시간가는줄 모르겠다 ㅋㅋ 노트5인데 배터리 광탈이 아쉬움 ㅜ
교정 이후 len = 33 
 시간가는줄 모르겠다 ㅋㅋ 노트5인데 배터리 광탈이 아쉬움 ㅜ
----------------------
교정 이전 len = 71 
 다운을 받으려했는데 리소스파일? 그거16/22까지 다운받고 멈추더군요 10분정도를 기다려도 그상태그대로고요 이거어떻게해야하나요?
교정 이후 len = 74 
 다운을 받으려했는데 리소스파일? 그거16/22까지 다운받고 멈추더군요 10 분정도를 기다려도 그상태그대로고요 이거 어떻게 해야하나요?
----------------------
교정 이전 len = 97 
 뿌우랑 펭귄인지 삿는데 시공석만 빠져나갔다는 저사람은

----------------------
교정 이전 len = 38 
 인터넷 잘 되는데 인터넷이 않된다고 접속할수 없다네 희한하네 버그인가
교정 이후 len = 38 
 인터넷 잘 되는데 인터넷이 않된다고 접속할수 없다네 희한하네 버그인가
----------------------
교정 이전 len = 55 
 아직 챔피언수가 적고 모델링도 체형별로 평타가 다 똑같아서 개성이 없음=여러 챔피언이 많이 필요없음
교정 이후 len = 55 
 아직 챔피언수가 적고 모델링도 체형별로 평타가 다 똑같아서 개성이 없음=여러 챔피언이 많이 필요없음
----------------------
교정 이전 len = 29 
 잼 좀 마니주세요 재미있는데 현질안하고는 하기힘들어요
교정 이후 len = 29 
 잼 좀 마니주세요 재미있는데 현질안하고는 하기힘들어요
----------------------
교정 이전 len = 46 
 이거 게임을 들어갈려고 해도 들어가지지가 않네요...왜이러죠..?넘 복잡하네요...
교정 이후 len = 46 
 이거 게임을 들어갈려고 해도 들어가지지가 않네요...왜이러죠..?넘 복잡하네요...
----------------------
교정 이전 len = 38 
 오늘 화면이 안넘어가고.멈처서 게임을 즐길수없다 화면이 게임이 안됨.
교정 이후 len = 38 
 오늘 화면이 안넘어가고.멈처서 게임을 즐길수없다 화면이 게임이 안됨.
----------------------
교정 이전 len = 139 
 현질 외에는. 답이없는 게임. .. 중반부까지.신나서 현질해가며 하다가 ..지쳤다 뭐 정도껏이여야지. ... 열심히.재빠르게. 외워서! 하면 되겠지 했는데 .. 현질 업그레이드 안하면 별 3개는.커녕 패스도 힘든지경. 그간 현질은 아깝지만 그만하련다
교정 이후 len = 140 
 현질 외에는. 답이 없는 게임. .. 중반부까지.신나서 현질해가며 하다가 ..지쳤다 뭐 정도껏이여야지. ... 열심히.재빠르게. 외워서! 하면 되겠지 했는데 .. 현질 

In [93]:
# 추가적인 전처리

# 한글과 다른 무언가가 붙어있는경우를 띄워줌.
def spacing(string):
    string = re.sub('([가-힣])([^가-힣 ])','\g<1> \g<2>',string)
    string = re.sub('([^가-힣 ])([가-힣])','\g<1> \g<2>',string)
    return string

# 1 2 이런식으로 띄어져있는것을 12로 붙여줌.
def number_paste(string):
    return re.sub('([0-9]) *([0-9])','\g<1>\g<2>',string)

corrected_review = WHOLE_REVIEW_DF['교정된 리뷰']
corrected_review = corrected_review.apply(spacing)
corrected_review = corrected_review.apply(number_paste)

In [109]:
# 반복되는 문자가 아직도 잘 제거가 안되길래 반복되는 횟수를 최대 2회로 조정.
def repetition_remove(string):
    return re.sub(r'([^0-9])\1+', r'\1\1', string)

corrected_review = corrected_review.apply(repetition_remove)

In [117]:
len_compile = corrected_review.apply(lambda x : len(x) >= 10)

# 길이가 10이상인 글만 한번 프린트해봤음.
print(*corrected_review[len_compile].values[:500], sep='\n\n')

재미있게 잘하고있어요

서재훈님 혹시 이것을 볼지는 모르겠지만 훈련소 시스템은 예전엔 그랬지만 업뎃으로 인해 변경된 것입니다

제가 8 홀까지 하다 지웠는데 다시 게임 설치했더니 5 홀로 남아있네요

ㅍㅌ 하 ㅕ 카 ㅕ

이 게임 추천할 수 있을 만한 게임입니다 .

장인기지 매칭 시스템좀 개선해주세요 . 계속 +_ 2 홀까지 매칭 되는것으로 보이는데 계속 +2 홀들만 나오고 저보다 낮은 홀들은 보이지도 않네요 . 너무한것 아닙니까 ?

훈련소에서 2 개씩 풀리면 좋겠습니다 .

전략을 세우는 게임을 좋와 하는 편이라 재밋게 하고잇어요 .ㅎㅎ

슈퍼셀은 참 휼륭한 회사다

게임 잘하고 있습니다 ^^

오랜만에 이게임을 들어갔는데 재밌더라구요 강추 !!

키우는 재미와 이긴는 재미 덕분에 중복이 되어 계속하게됩니다 . 정말 재미있는 게임이에요 .

나중엔보석을 더 많이주시면 안될까요 ?

현질 필요하지만 최고의 게임이다

너무 재미있어요 .

많은 업데이트 , 이벤트로 많은 유저들에게 사랑을 받는 겜이 되었네요 . 요즘 정말 재밌게 플레이하고 있습니다 ^^ 근데 한가지 부탁하고 싶은 게 있는데요 , 클랜원들과 ' 도전 ' 버튼으로 서로 공격할 수 있잖아요 , 근데 건물을 이동시키면 제한시간이 생기더라고요 . 저는 장벽을 자주 업글을 하고 깔끔하게 보이기 위해 장벽을 같은 렙끼리 모아놓는데 , 이것 때문에 도전 모드에 제한시간이 생겨서 ' 도전 ' 을하고 싶어도 그럴 수가 없네요 그러니깐 장벽만 이동시키는 건 제한시간을 안받게 해주면 고맙겠어요 ~~

뭐야 별보너스 초기화시간 지났는데 왜 다시 21 시간으로 늘어났음 ??

너무 잼 중국인들이 자꾸 핵을 써서 못 하겠습니다 .

아니 .. 지금 별보너스 5 배라서 열심히 하고 잇는데 공격 가도 네트워크 문제로 별 안 쌓이고 그러네요 .. 열심히 클오클하는 유저인데 .. 이 부분은 고쳐주셔야할 듯 합니다

이게임을 만들어줘서 검사합니다

머무재미있어서개속하고싯다

하 ᆢ 보상제대로 좀 주세요 아니별 5 개모으면앨릭서

In [123]:
#corrected_review = corrected_review.reset_index().drop('index',axis=1)
WHOLE_REVIEW_DF['교정된 리뷰'] = corrected_review
WHOLE_REVIEW_DF = WHOLE_REVIEW_DF.reset_index().drop('index',axis=1)

WHOLE_REVIEW_DF.to_excel('corrected_whole_review.xlsx')